In [101]:
import io
import requests
import PyPDF2
import re
import spacy
import gensim
import wordcloud
import mlflow
import sklearn
import string
import pandas as pd
import json

In [102]:
def extract_content(url):
    try: 
        response = requests.get(url)
        open_pdf_file = io.BytesIO(response.content)
        pdf = PyPDF2.PdfReader(open_pdf_file)
        
        # Access pdf content 
        text = []
        for i in range(len(pdf.pages)):
            page = pdf.pages[i]
            text.append(page.extract_text())
        
        # Return concatenated content 
        return "\n".join(text)
    except Exception as e:
        print(f"Error occurred while processing {url}: {e}")
        return ""

In [103]:
"""
def clean_esg_report_urls(urls):
    cleaned_urls = []
    for url in set(urls):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                cleaned_urls.append(url)
        except:
            pass
    return cleaned_urls
cleaned_urls = clean_esg_report_urls(urls)
print(cleaned_urls)
"""

'\ndef clean_esg_report_urls(urls):\n    cleaned_urls = []\n    for url in set(urls):\n        try:\n            response = requests.get(url)\n            if response.status_code == 200:\n                cleaned_urls.append(url)\n        except:\n            pass\n    return cleaned_urls\ncleaned_urls = clean_esg_report_urls(urls)\nprint(cleaned_urls)\n'

In [104]:
esg_urls={ 
  'barclays': 'https://www.responsibilityreports.com/HostedData/ResponsibilityReports/PDF/LSE_BARC_2021.pdf',
  'jp morgan chase': 'https://www.jpmorgan.com/content/dam/jpmc/jpmorgan-chase-and-co/documents/jpmc-esg-report-2021.pdf',
  'BNP Paribas': 'https://cdn-group.bnpparibas.com/uploads/file/bnpp_climateanalytics_alignmentreport_final.pdf',
  'goldman sachs': 'https://www.goldmansachs.com/a/2021-sustainability-report.pdf',
  'hsbc': 'https://www.hsbc.com/-/files/hsbc/investors/hsbc-results/2021/annual/pdfs/hsbc-holdings-plc/220222-esg-review-2021.pdf',
  'PNC':'https://www.pnc.com/content/dam/pnc-com/pdf/aboutpnc/CorporateResponsibilityReports/PNC_Corporate_Responsibility_Report_2021.pdf',
  'bank of america': 'https://about.bankofamerica.com/content/dam/about/pdfs/ESG_GHI_2021_508_secured.pdf',
  'rbc': 'https://www.rbc.com/community-social-impact/_assets-custom/pdf/2021-ESG-Report.PDF',
  'macquarie': 'https://www.macquarie.com/assets/macq/investor/reports/2021/sections/macquarie-group-fy21-esg.pdf',
  'lloyds': 'https://assets.lloyds.com/media/8c362b67-e4a5-4876-927f-397c10491d72/Lloyds_ESG%202021_report_final.pdf',
  'santander': 'https://www.santander.com/content/dam/santander-com/en/contenido-paginas/nuestro-compromiso/reports/doc-informe-BR-polonia-2021.pdf',
  'bluebay': 'https://www.rbcbluebay.com/globalassets/documents/bluebay-esg-investment-update-june-2021.pdf',
  'riverstone': 'https://www.riverstonellc.com/media/1328/riverstone-esg-report-2021_updated.pdf',
  'aberdeen standard': 'https://www.abrdn.com/docs?documentId=GB-080322-167278-5',
  'apollo': 'https://www.apollo.com/~/media/Files/A/Apollo-V3/documents/apollo-2021-esg-report-final.pdf',
  'vanguard': 'https://www.nl.vanguard/content/dam/intl/europe/documents/en/climate-change-tcfd-report_en-eu.pdf',
  'ruffer': 'https://www.ruffer.co.uk/-/media/ruffer-website/files/downloads/cat/2021-cat-responsible-investment-report.pdf?la=en&hash=2EB1AE3E432C88D6E51015A3079A5D09',
         }
# Prepare PDF into Dataframe
esg_urls_pd = pd.DataFrame(list(esg_urls.items()), columns=['company', 'url'])

esg_urls = pd.DataFrame(esg_urls_pd)

In [105]:
esg_urls

,company,url
0,barclays,https://www.responsibilityreports.com/HostedDa...
1,jp morgan chase,https://www.jpmorgan.com/content/dam/jpmc/jpmo...
2,BNP Paribas,https://cdn-group.bnpparibas.com/uploads/file/...
3,goldman sachs,https://www.goldmansachs.com/a/2021-sustainabi...
4,hsbc,https://www.hsbc.com/-/files/hsbc/investors/hs...
5,PNC,https://www.pnc.com/content/dam/pnc-com/pdf/ab...
6,bank of america,https://about.bankofamerica.com/content/dam/ab...
7,rbc,https://www.rbc.com/community-social-impact/_a...
8,macquarie,https://www.macquarie.com/assets/macq/investor...
9,lloyds,https://assets.lloyds.com/media/8c362b67-e4a5-...


In [106]:
# add a new column to the data frame esg_urls_pd with the name 'content'

esg_urls_pd['content'] = esg_urls_pd['url'].apply(extract_content)
esg_articles = esg_urls_pd[esg_urls_pd['content'].str.len() > 0] # that only includes rows with non-empty content

Error occurred while processing https://www.responsibilityreports.com/HostedData/ResponsibilityReports/PDF/LSE_BARC_2021.pdf: EOF marker not found
Error occurred while processing https://www.pnc.com/content/dam/pnc-com/pdf/aboutpnc/CorporateResponsibilityReports/PNC_Corporate_Responsibility_Report_2021.pdf: EOF marker not found
Error occurred while processing https://about.bankofamerica.com/content/dam/about/pdfs/ESG_GHI_2021_508_secured.pdf: PyCryptodome is required for AES algorithm
Error occurred while processing https://www.rbcbluebay.com/globalassets/documents/bluebay-esg-investment-update-june-2021.pdf: EOF marker not found
Error occurred while processing https://www.apollo.com/~/media/Files/A/Apollo-V3/documents/apollo-2021-esg-report-final.pdf: EOF marker not found


In [107]:
esg_articles

,company,url,content
1,jp morgan chase,https://www.jpmorgan.com/content/dam/jpmc/jpmo...,2021 \nENVIRONMENTAL \nSOCIAL & \nGOVERNAN...
2,BNP Paribas,https://cdn-group.bnpparibas.com/uploads/file/...,1\nBNP PARIBAS / CLIMATE ANALYTICS AND ALIGNM...
3,goldman sachs,https://www.goldmansachs.com/a/2021-sustainabi...,Progress \nThrough \nPerformanceSUSTAINABILITY...
4,hsbc,https://www.hsbc.com/-/files/hsbc/investors/hs...,"Environmental, social \nand governance review\..."
7,rbc,https://www.rbc.com/community-social-impact/_a...,HNW_NRG_B_Bleed_NoMask\nRoyal Bank of Canada\n...
8,macquarie,https://www.macquarie.com/assets/macq/investor...,"52\nEnvironmental, Social and Governance\nMacq..."
9,lloyds,https://assets.lloyds.com/media/8c362b67-e4a5-...,"Environmental, Social \nand Governance \nRepor..."
10,santander,https://www.santander.com/content/dam/santande...,ESG Report 2021\nLooking to the future\nRespon...
12,riverstone,https://www.riverstonellc.com/media/1328/river...,Investing \nResponsibly\nRiverstone ESG Repor...
13,aberdeen standard,https://www.abrdn.com/docs?documentId=GB-08032...,The Power \nof Investment\nSustainability Rep...


In [108]:
def remove_non_ascii(text):
    """
    Removes non-ASCII characters from the given text
    """
    printable = set(string.printable)
    return ''.join(filter(lambda x: x in printable, text)) 
#The filter() function is then used with a lambda function to iterate over each character in the input text and keep only those characters that are present in the printable set. 
#join() method to concatenate the filtered characters back into a single string, effectively removing any non-ASCII characters.

In [109]:
def not_header(line):
                                           # as we're consolidating broken lines into paragraphs, we want to make sure not to include headers
  return not line.isupper()

In [110]:
nlp = spacy.load('en_core_web_sm')

#When you load the 'en_core_web_sm' model using the spacy.load() function, you are essentially loading a trained NLP model into your Python code, which can then be used to process text data and perform various NLP tasks. Once the model is loaded, you can use its functionalities, such as tokenization, sentence segmentation, entity recognition, and more, to analyze and extract information from text data in English.

In [111]:
def extract_statements(nlp, text):
    """
    Extracts ESG statements from raw text by removing junk, URLs, etc.
    Groups consecutive lines into paragraphs and uses Spacy to parse sentences.
    """
    # remove non ASCII characters
    text = remove_non_ascii(text)

    lines = []
    prev = ""
    for line in text.split('\n'):
        # aggregate consecutive lines where text may be broken down
        # only if next line starts with a space or previous does not end with dot.
        if(line.startswith(' ') or not prev.endswith('.')):
            prev = prev + ' ' + line
        else:
            # new paragraph
            lines.append(prev)
            prev = line

    # don't forget left-over paragraph
    lines.append(prev)

    # clean paragraphs from extra space, unwanted characters, urls, etc.
    # best effort clean up, consider a more versatile cleaner
    sentences = []
    for line in lines:

        # removing header number
        line = re.sub(r'^\s?\d+(.*)$', r'\1', line)
        # removing trailing spaces
        line = line.strip()
        # words may be split between lines, ensure we link them back together
        line = re.sub('\s?-\s?', '-', line)
        # remove space prior to punctuation
        line = re.sub(r'\s?([,:;.])', r'\1', line)
        # ESG contains a lot of figures that are not relevant to grammatical structure
        line = re.sub(r'\d{5,}', r' ', line)
        # remove mentions of URLs
        line = re.sub(r'((http|https)://)?[a-zA-Z0-9./?:@-=#]+.([a-zA-Z]){2,6}([a-zA-Z0-9.&/?:@-_=#])*', r' ', line)
        # remove multiple spaces
        line = re.sub('\s+', ' ', line)

        # split paragraphs into well defined sentences using Spacy
        for part in list(nlp(line).sents):
            sentences.append(str(part).strip())

    return sentences

In [112]:
esg_articles = esg_urls.copy()
esg_articles['content'] = esg_articles['url'].apply(lambda url: extract_content(url))
esg_articles = esg_articles[esg_articles['content'].str.len() > 0] 

Error occurred while processing https://www.responsibilityreports.com/HostedData/ResponsibilityReports/PDF/LSE_BARC_2021.pdf: EOF marker not found
Error occurred while processing https://www.pnc.com/content/dam/pnc-com/pdf/aboutpnc/CorporateResponsibilityReports/PNC_Corporate_Responsibility_Report_2021.pdf: EOF marker not found
Error occurred while processing https://about.bankofamerica.com/content/dam/about/pdfs/ESG_GHI_2021_508_secured.pdf: PyCryptodome is required for AES algorithm
Error occurred while processing https://www.rbcbluebay.com/globalassets/documents/bluebay-esg-investment-update-june-2021.pdf: EOF marker not found
Error occurred while processing https://www.apollo.com/~/media/Files/A/Apollo-V3/documents/apollo-2021-esg-report-final.pdf: EOF marker not found


In [113]:
esg_articles

,company,url,content
1,jp morgan chase,https://www.jpmorgan.com/content/dam/jpmc/jpmo...,2021 \nENVIRONMENTAL \nSOCIAL & \nGOVERNAN...
2,BNP Paribas,https://cdn-group.bnpparibas.com/uploads/file/...,1\nBNP PARIBAS / CLIMATE ANALYTICS AND ALIGNM...
3,goldman sachs,https://www.goldmansachs.com/a/2021-sustainabi...,Progress \nThrough \nPerformanceSUSTAINABILITY...
4,hsbc,https://www.hsbc.com/-/files/hsbc/investors/hs...,"Environmental, social \nand governance review\..."
7,rbc,https://www.rbc.com/community-social-impact/_a...,HNW_NRG_B_Bleed_NoMask\nRoyal Bank of Canada\n...
8,macquarie,https://www.macquarie.com/assets/macq/investor...,"52\nEnvironmental, Social and Governance\nMacq..."
9,lloyds,https://assets.lloyds.com/media/8c362b67-e4a5-...,"Environmental, Social \nand Governance \nRepor..."
10,santander,https://www.santander.com/content/dam/santande...,ESG Report 2021\nLooking to the future\nRespon...
12,riverstone,https://www.riverstonellc.com/media/1328/river...,Investing \nResponsibly\nRiverstone ESG Repor...
13,aberdeen standard,https://www.abrdn.com/docs?documentId=GB-08032...,The Power \nof Investment\nSustainability Rep...


In [114]:
def extract_statements_udf(content_series, nlp):

    statements = []
    for content in content_series:
        # remove non ASCII characters
        text = remove_non_ascii(content)

        # split into paragraphs and sentences
        lines = []
        prev = ""
        for line in text.split('\n'):
            if(line.startswith(' ') or not prev.endswith('.')):
                prev = prev + ' ' + line
            else:
                lines.append(prev)
                prev = line
        lines.append(prev)

        # clean paragraphs from extra space, unwanted characters, urls, etc.
        # best effort clean up, consider a more versatile cleaner
        for line in lines:
            line = re.sub(r'^\s?\d+(.*)$', r'\1', line)  # removing header number
            line = line.strip()  # removing trailing spaces
            line = re.sub('\s?-\s?', '-', line)  # words may be split between lines, ensure we link them back together
            line = re.sub(r'\s?([,:;.])', r'\1', line)  # remove space prior to punctuation
            line = re.sub(r'\d{5,}', r' ', line)  # remove numbers with more than 5 digits
            line = re.sub(r'((http|https)://)?[-a-zA-Z0-9./?:@=#]+\\.([a-zA-Z]){2,6}([a-zA-Z0-9.&/?:@-_=#])*', r' ', line)
            line = re.sub('\s+', ' ', line)  # remove multiple spaces

            # split paragraphs into sentences using spacy
            for part in list(nlp(line).sents):
                sentence = str(part).strip()
                if len(sentence) > 100:  # filter out short sentences
                    statements.append(sentence)

    return statements

In [115]:
tmp = pd.DataFrame(columns=["company", "statements"])
for company, content in esg_articles[["company", "content"]].values:
    statements = extract_statements_udf(content_series=pd.Series([content]), nlp=nlp)
    tmp = tmp.append({"company": company, "statements": statements}, ignore_index=True)

esg_statements = tmp.explode(column="statements").reset_index(drop=True)

In [116]:
esg_statements

,company,statements
0,jp morgan chase,2021 ENVIRONMENTAL SOCIAL & GOVERNANCE REPORT ...
1,jp morgan chase,Feature: Our Commitment to Racial Equity 21Div...
2,jp morgan chase,It has also shown what companies like ours can...
3,jp morgan chase,"Throughout this period of uncertainty, JPMorga..."
4,jp morgan chase,We are leveraging capital and expertise across...
...,...,...
8614,ruffer,This financial promotion is issued by Ruffer L...
8615,ruffer,"Although Ruffer LLPs information providers, in..."
8616,ruffer,None of the ESG Parties makes any express or i...
8617,ruffer,None of the ESG Parties shall have any liabili...


In [117]:
esg_statements.to_csv('esg_statements.txt', sep='\t', index=False, header=False)


In [118]:
def lemmatize(text):
    # Parse text using spacy
    doc = nlp(text)

    # Convert words into their simplest form (singular, present form, etc.)
    lemma = []
    for token in doc:
        if token.lemma_ != '-PRON-':
            lemma.append(token.lemma_)
    return ' '.join(lemma)



#This version of the function uses token.lemma_ instead of token.lemma to access the lemmatized form of the token, and returns a string instead of a tokenized string.


In [119]:
def tokenize(sentence):
    gen = gensim.utils.simple_preprocess(sentence, deacc=True) #performs basic text preprocessing, including lowercasing, tokenization, and removing punctuation marks
    return ' '.join(gen)

In [ ]:

# Apply the lemmatize function to each statement in the DataFrame
esg_statements['lemma'] = esg_statements['statements'].apply(lemmatize)

# Select only the columns we want to keep
esg_lemma = esg_statements[['company', 'statements', 'lemma']]


In [ ]:
esg_lemma

In [ ]:
esg_lemma.to_csv('esg_lemma.txt', sep='\t', index=False, header=False)


In [ ]:
def remove_stop_words_and_company_names(lemma, company_name):
    # Parse lemma using spacy
    doc = nlp(lemma)
    extra=['plc', 'group', 'target',
           'track', 'capital', 'holding',
           'report', 'annualreport',
           'esg', 'bank', 'report',
           'annualreport', 'long', 'make','2021 tcfd','2020 2021','https www',
          ]
    # Create a list of stop words and company names
    stop_words = spacy.lang.en.stop_words.STOP_WORDS
    company_name_tokens = company_name.split()                               # tokenize the company name
    stop_words.update(company_name_tokens)
    stop_words.update(extra)


    # Remove stop words and company names from the lemma
    filtered_tokens = []
    for token in doc:
        if token.text.lower() not in stop_words:
            filtered_tokens.append(token.text)

    # Join the filtered tokens back into a string
    filtered_lemma = ' '.join(filtered_tokens)
    return filtered_lemma

In [ ]:
# Apply the remove_stop_words_and_company_names() function to 'lemma' column in 'esg_statements' DataFrame
esg_statements['lemma'] = esg_statements.apply(lambda row: remove_stop_words_and_company_names(row['lemma'],
                                                                                               row['company']), axis=1)
esg_lemmatized=esg_statements[['company', 'statements', 'lemma']]

In [ ]:
esg_lemmatized

In [ ]:
esg_lemmatized.to_csv('esg_lemmatized.txt', sep='\t', index=False, header=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Define stop words
stop_words = set(stopwords.words('english'))
stop_words = list(stop_words)



In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


large_string = ' '.join(esg_lemmatized.lemma)

# use 3rd party lib to compute term freq., apply stop words
word_cloud = WordCloud(
    background_color="white",
    max_words=5000, 
    width=900, 
    height=700, 
    stopwords=stop_words, 
    contour_width=3, 
    contour_color='steelblue'
)

# display our wordcloud across all records
plt.figure(figsize=(10,10))
word_cloud.generate(large_string)
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()


In [ ]:
# Run bi-gram TF-IDF frequencies
bigram_tf_idf_vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(2,2), min_df=10, use_idf=True)
bigram_tf_idf = bigram_tf_idf_vectorizer.fit_transform(esg_lemmatized['lemma'])


In [ ]:
bigram_tf_idf

In [ ]:
#display bigram_tf_idf 
# Convert bigram_tf_idf to dense matrix
dense_matrix = bigram_tf_idf.todense()

# Print dense matrix
print(dense_matrix)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

words = list(bigram_tf_idf_vectorizer.vocabulary_.keys())

# extract our top 10 ngrams
total_counts = np.zeros(len(words))
for t in bigram_tf_idf:
    total_counts += t.toarray()[0]
count_dict = (zip(words, total_counts))
count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]     #sliced using [0:10] to select only the top 10 entries with the highest total counts.
words = [w[0] for w in count_dict]
counts = [w[1] for w in count_dict]
x_pos = np.arange(len(words)) 



In [ ]:
words

In [ ]:
total_counts

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Plot top 10 ngrams
plt.figure(figsize=(15, 5))
plt.subplot(title='10 most common bi-grams')
sns.barplot(x=words, y=counts, palette='Blues_r')
plt.xticks(x_pos, words, rotation=90) 
plt.xlabel('bi-grams')
plt.ylabel('tfidf')
plt.show()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA


In [ ]:
mlflow.end_run()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

from sklearn.feature_extraction.text import CountVectorizer

#Topic modelling
word_tf_vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1,1))                    #counting the number of occurrences of each word in each document.
word_tf = word_tf_vectorizer.fit_transform(esg_lemmatized.lemma)
# serialize stopwords 
json_data = json.dumps([a for a in stop_words], indent=2)
f = open("C:\\Users\\User\\ESG\\stopwords.json", "w")
f.write(json_data)
f.close()
  
# track experiment on MLflow
with mlflow.start_run(run_name='topic_modelling'):                                            #how many args could we pass ?
  
  # Train a LDA model with 9 topics
  lda = LDA(random_state = 42, n_components = 9, learning_decay = .3)
  lda.fit(word_tf) # trains the LDA model on the word frequency matrix word_tf.

In [ ]:
  # Log model 
  mlflow.sklearn.log_model(lda, "model")
  mlflow.log_param('n_components', '9')
  mlflow.log_param('learning_decay', '.3')
  mlflow.log_metric('perplexity', lda.perplexity(word_tf))
  mlflow.log_artifact("C:\\Users\\User\\ESG\\stopwords.json")
  

In [ ]:
  # retrieve run ID to attach topic name later
  lda_run_id = mlflow.active_run().info.run_id

In [ ]:
# Retrieve top N words describing each of our 9 topics

def top_words(model, feature_names, n_top_words):
    topics_words = []
    for topic_idx, topic in enumerate(model.components_):
        
        words = []
        for i in topic.argsort()[:-n_top_words-1:-1]:
            if len(feature_names[i]) > 1:
                words.append(feature_names[i])
            else:
                words.append('')
            
        topics_words.append(words)
        df = pd.DataFrame(topics_words)
    return df



In [ ]:
# Display topics keywords for inspection and for us to label topics
# Vectorize words using TF-IDF Vectorizer

tf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tf_vectorizer.fit(esg_lemmatized.lemma)
tf_feature_names = tf_vectorizer.get_feature_names_out()
display(top_words(lda, tf_feature_names, 15))


In [ ]:
#Define topic names
# We estimated our topics to be described around the following themes

topic_names = [
    'green energy',
    'focus customer',
  'value employees',
  'strong governance', 
  'support community',
  'company transformation',
  'ethical investments',
  'sustainable finance',
  'code of conduct',
]



In [ ]:
# serialize topic names
json_data = json.dumps(topic_names, indent=2)
f = open("C:\\Users\\User\\ESG\\topics.json", "w")
f.write(json_data)
f.close()

In [ ]:
# attach topics name to LDA model on mlflow
# we do not wish to re-open run_id and therefore alter start / end time
# instead, we log artifact to existing run
client = mlflow.tracking.MlflowClient()
client.log_artifact(lda_run_id, "C:\\Users\\User\\ESG\\topics.json")

In [ ]:
from wordcloud import WordCloud

In [ ]:
#Display topics
# We ensure relevance of our topics using simple wordcloud visualisation
from wordcloud import WordCloud

def word_cloud(model, tf_feature_names, index):
    
    imp_words_topic=""
    comp = model.components_[index]
    vocab_comp = zip(tf_feature_names, comp)
    sorted_words = sorted(vocab_comp, key = lambda x:x[1], reverse=True)[:50]
    
    for word in sorted_words:
        imp_words_topic = imp_words_topic + " " + word[0]
    
    return WordCloud(
        background_color="white",
        width=600, 
        height=600, 
        contour_width=3, 
        contour_color='steelblue'
    ).generate(imp_words_topic)
    
topics = len(lda.components_)
fig = plt.figure(figsize=(20, 20 * topics / 3))

# Display wordcloud for each extracted topic
for i, topic in enumerate(lda.components_):
    ax = fig.add_subplot(topics,3, i + 1)
    ax.set_title(topic_names[i], fontsize=20)
    wordcloud = word_cloud(lda, tf_feature_names, i)
    ax.imshow(wordcloud)
    ax.axis('off')
    

In [ ]:
#Attach topic distribution to each ESG statement
# score our original dataset to attach topic distribution to each ESG statement
transformed = lda.transform(word_tf)

# find principal topic from distribution...
a = [topic_names[np.argmax(distribution)] for distribution in transformed]

# ... with associated probability
b = [np.max(distribution) for distribution in transformed]

# consolidate LDA output into a handy dataframe 
df1 = esg_lemmatized[['company', 'statements', 'lemma']]
df2 = pd.DataFrame(zip(a,b,transformed), columns=['topic', 'probability', 'probabilities'])
esg_group = pd.concat([df1, df2], axis=1)

# display dataframe
display(esg_group[['company', 'lemma', 'topic', 'probability']])

In [ ]:
#Attach topic distribution to each ESG statement
# score our original dataset to attach topic distribution to each ESG statement
transformed = lda.transform(word_tf)

# find principal topic from distribution...
a = [topic_names[np.argmax(distribution)] for distribution in transformed]

# ... with associated probability
b = [np.max(distribution) for distribution in transformed]

# consolidate LDA output into a handy dataframe 
df1 = esg_lemmatized[['company', 'lemma', 'statements']]
df2 = pd.DataFrame(zip(a,b,transformed), columns=['topic', 'probability', 'probabilities'])
esg_group = pd.concat([df1, df2], axis=1)

# display dataframe
display(esg_group[['company', 'lemma', 'topic', 'probability']])

In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
#Compare companies core ESG initiatives
# create a simple pivot table of number of occurence of each topic across organisations

esg_focus = pd.crosstab(esg_group.company, esg_group.topic)

# scale topic frequency between 0 and 1
scaler = MinMaxScaler(feature_range = (0, 1))

# normalize pivot table
esg_focus_norm = pd.DataFrame(scaler.fit_transform(esg_focus), columns=esg_focus.columns)
esg_focus_norm.index = esg_focus.index

In [ ]:
esg_focus_norm

In [ ]:
esg_focus

In [ ]:
# plot heatmap, showing main area of focus for each FSI across topics we learned
sns.set(rc={'figure.figsize':(10,5)})
sns.heatmap(esg_focus_norm, annot=False, linewidths=.5, cmap='Blues')
plt.show()

In [ ]:
#Show distribution of topic probability

# not every statement follows a well defined topic

# some statements may be more generic and span across multiple themes

esg_group.probability.hist(bins=50, figsize=(8,5), color='steelblue')

# plot distribution of main topic proability
plt.axvline(0.89, color='coral', linestyle='--')
plt.title('Primary topic distribution')
plt.xlabel('probability')
plt.ylabel('density')
plt.show()

In [ ]:
esg_group.probability

In [ ]:
#Retrieve key statements for each topic

# extract statements relevant to a given topic

topic_discussions = esg_group[esg_group['topic'] == 'green energy']

# as specified in probability distribution, we want only specific topics and not general discussions
topic_discussions = topic_discussions[topic_discussions['probability'] > 0.89]

# access more specific topics first

topic_discussions = topic_discussions.sort_values('probability', ascending=False)

rows = [] 
for i, row in topic_discussions.iterrows():
  rows.append([row.company, row.probability, row.statements])

# display dataframe of statements for selected topic of interest
display(pd.DataFrame(rows, columns=['company', 'probability', 'statement']))

In [ ]:
#STEP3: Key ESG initiatives
#identifier les declarations spécific à chaque entreprise à partir des proba de LDA en utilisant KMeans : clustering chaque declaration en fct de leur similitude

In [ ]:
from sklearn.cluster import KMeans


In [ ]:
#1 déterminier le nombre optimale des clusters K avec ELBOW METHOD
# we extract our probabilities distribution as input vectors for KMeans
X_train = list(esg_group.probabilities)

# nevertheless, we still want to ensure relavance of our clustering using the simple "elbow method"
# we capture the sum of squared distance of each point to their closest center for different values of k
wsses = []
for k in [5, 8, 10, 20, 30, 50, 80, 100]:
  kmeans = KMeans(n_clusters=k, random_state=42)
  kmeans.fit(X_train)
  wsse = np.sum([np.min(x)**2 for x in kmeans.transform(X_train)]) 
  wsses.append([k, wsse])
  
#we simply plot the WSSE against K to find optimal K value
wsse_df = pd.DataFrame(wsses, columns=['k', 'wsse'])
display(wsse_df)



In [ ]:
import matplotlib.pyplot as plt

# plot the WSSE against K to find optimal K value
plt.plot(wsse_df['k'], wsse_df['wsse'])
plt.title('Elbow Method')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Sum of Squared Distance')
plt.show()


In [ ]:
# K is aroud 15 - 20 cluster

In [ ]:
mlflow.end_run()

In [ ]:
#grouper les déclarations en clusters et déterminer la plus proche distance
# track experiment on MLflow
with mlflow.start_run(run_name='clustering'):
  
  # we train our KMeans model trained with the appropriate value for K
  kmeans = KMeans(n_clusters=20, random_state=42)
  kmeans.fit(X_train)
  
  # Log model 
  mlflow.sklearn.log_model(kmeans, "model")
  mlflow.log_param('n_clusters', '15')
  mlflow.log_metric('wsse', np.sum([np.min(x)**2 for x in kmeans.transform(X_train)]))
  
  # Retrieve experiment ID
  cluster_run_id = mlflow.active_run().info.run_id

In [ ]:
# Assign statements to closest clusters
# find the minimum distance for each point to their closest cluster
y_dist = [np.min(x) for x in kmeans.transform(X_train)]                          #calcule la distance minimale entre chaque point
dist_df = pd.DataFrame(zip(y_dist), columns=['distance'])
esg_group_dist = pd.concat([esg_group, dist_df], axis=1)

In [ ]:
esg_group_dist[['company', 'lemma', 'topic','probability','distance']]

In [ ]:
esg_group_dist

In [ ]:
esg_group_dist.to_csv('esg_reports.csv', index=False)

In [ ]:
!pip install delta-spark

In [ ]:
!pip install pyspark


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

from sqlalchemy import create_engine, text

spark = SparkSession.builder.appName("NLP_ESG").getOrCreate()

df = pd.DataFrame(esg_group_dist)
# create a SQLite engine
engine = create_engine('sqlite:///esg.db', echo=False)

df.to_sql('reports', con=engine, if_exists='replace')

In [ ]:
engine 

In [ ]:
from sqlalchemy import create_engine, text

query = """
    SELECT 
        t.company,
        t.topic,
        t.statement
       
    FROM (
        SELECT 
            e.company,
            e.topic,
            e.probability,
            e.distance,
            LOWER(e.statements) AS statement,
            dense_rank() OVER (PARTITION BY e.company, e.topic ORDER BY e.distance DESC) as rank
        FROM reports e
    ) t
    WHERE t.rank = 1
    AND t.topic IN ('green energy')
    ORDER BY company, topic, rank
"""

df = pd.read_sql_query(sql=query, con=engine.connect())


In [ ]:
df

In [ ]:
query = """
WITH ranked AS (
  SELECT 
    e.topic, 
    e.statements, 
    e.company,
    dense_rank() OVER (PARTITION BY e.company, e.topic ORDER BY e.probability DESC) as rank
  FROM reports e
)

SELECT 
  t.topic,
  t.statements
FROM ranked t
WHERE t.company = 'BNP Paribas' 
AND t.rank = 1
"""
df = pd.read_sql_query(sql= text(query), con=engine.connect())

In [ ]:
df

In [ ]:
import pandas as pd


# Sélectionner les colonnes pertinentes
relevant_columns = ["topic", "company", "probability"]
esg_scores = esg_group_dist[relevant_columns]

# Grouper les données par "topic" et "company" et calculer la somme des probabilités pour chaque groupe
esg_scores = esg_scores.groupby(["topic", "company"]).agg(esg=("probability", "sum")).reset_index()

# Ajouter une colonne "rank" représentant le classement des organisations en fonction de "esg", en les ordonnant par ordre croissant
esg_scores["rank"] = esg_scores["esg"].rank(ascending=True)

# Calculer le score ESG en divisant le classement par le nombre total d'organisations et en le multipliant par 100
esg_scores["esg"] = (esg_scores["rank"] / len(esg_scores)) * 100

# Arrondir le résultat du calcul précédent à un entier
esg_scores["esg"] = esg_scores["esg"].round().astype(int)

# Sélectionner les colonnes "company", "topic" et "esg" pour les résultats finaux
esg_score = esg_scores[["company", "topic", "esg"]]




In [ ]:
import pandas as pd


relevant_columns = ["topic", "company", "probability"]
esg_scores = esg_group_dist[relevant_columns]

esg_scores = esg_scores.groupby(["topic", "company"]).agg(esg=("probability", "sum")).reset_index()

esg_scores["rank"] = esg_scores["esg"].rank(ascending=True)

esg_scores["esg"] = (esg_scores["rank"] / len(esg_scores)) * 100

esg_scores["esg"] = esg_scores["esg"].round().astype(int)

esg_score = esg_scores[["company", "topic", "esg"]]




In [ ]:
esg_score

In [ ]:
esg_score.info()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Transposer les données pour avoir les entreprises en index et les topics en colonnes
esg_score_pivot = esg_score.pivot(index='company', columns='topic', values='esg')
# Tracer un graphique en barres pour chaque entreprise et chaque topic
esg_score_pivot.plot(kind='bar')

# Ajouter des décorations à la charte
plt.xlabel('Entreprise', fontsize=12)
plt.ylabel('Score ESG', fontsize=12)
plt.title('Scores ESG par entreprise et par topic', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Topic')

# Afficher la charte
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Tri des scores des entreprises pour chaque topic en ordre croissant
esg_score_sorted = esg_score.sort_values(by='esg')

# Création du graphique interactif
fig = go.Figure()

# Boucle sur chaque topic
for topic in esg_score['topic'].unique():
    # Filtrage des scores pour le topic spécifique
    scores_topic = esg_score_sorted[esg_score_sorted['topic'] == topic]
    
    # Ajout d'une trace pour le topic
    fig.add_trace(go.Bar(
        x=scores_topic['company'],
        y=scores_topic['esg'],
        name=topic
    ))

# Ajout des décorations au graphique
fig.update_layout(
    title='Scores ESG des entreprises par topic (tri croissant)',
    xaxis=dict(title='Entreprise'),
    yaxis=dict(title='Score ESG'),
    barmode='group',
    legend_title='Topic',
    width=1300,
    height=600
)

# Affichage du graphique interactif
fig.show()


In [ ]:
import seaborn as sns

# Transposer les données pour avoir les entreprises en index et les topics en colonnes
esg_score_pivot = esg_score.pivot(index='company', columns='topic', values='esg')

# Tracer une heatmap pour visualiser les scores ESG par entreprise et par topic
plt.figure(figsize=(10, 6))
sns.heatmap(esg_score_pivot, cmap='Blues', annot=True, fmt="d")

# Ajouter des décorations à la charte
plt.xlabel('Topic', fontsize=12)
plt.ylabel('Entreprise', fontsize=12)
plt.title('Scores ESG par entreprise et par topic', fontsize=14)
plt.xticks(rotation=45, ha='right')

# Afficher la charte
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Transposer les données pour avoir les entreprises en index et les topics en colonnes
esg_score_pivot = esg_score.pivot(index='company', columns='topic', values='esg')

# Créer une figure de heatmap interactive avec Plotly
fig = go.Figure(data=go.Heatmap(
    z=esg_score_pivot.values,
    x=esg_score_pivot.columns,
    y=esg_score_pivot.index,
    colorscale='Blues',
    hovertemplate='Entreprise: %{y}<br>Topic: %{x}<br>Score ESG: %{z}<extra></extra>'
))

# Ajouter des décorations à la figure
fig.update_layout(
    title='Scores ESG par entreprise et par sujet',
    xaxis=dict(title='Topic'),
    yaxis=dict(title='Entreprise')
)

# Afficher la figure interactive
fig.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Créer une copie du DataFrame esg_scores
esg_scores_copy = esg_scores.copy()

# Grouper les "topic" par "company" et calculer la moyenne de la colonne 'esg' pour chaque groupe
esg_scores_copy = esg_scores_copy.groupby("company").agg(esg=("esg", "mean")).reset_index()

# Ajouter une colonne 'sum' contenant la somme des scores agrégés
esg_scores_copy['sum'] = esg_scores_copy['esg'].sum()

# Définir l'index du DataFrame sur la colonne 'company'
esg_scores_copy = esg_scores_copy.set_index('company')

# Trier les données selon la colonne 'sum' par ordre décroissant
esg_scores_copy = esg_scores_copy.sort_values('sum', ascending=False)

# Supprimer la colonne 'sum' du DataFrame
esg_scores_copy = esg_scores_copy.drop('sum', axis=1)

# Tracer un graphique en barres des scores ESG par entreprise
colors = ["#3366cc", "#dc3912", "#ff9900", "#109618", "#990099"]
esg_scores_copy.plot(kind='bar', legend=False, color=colors)

plt.xlabel('Entreprise', fontsize=12)
plt.ylabel('Score ESG', fontsize=12)
plt.title('Scores ESG par entreprise', fontsize=14)
plt.xticks(rotation=45, ha='right')

# Afficher la charte
plt.tight_layout()
plt.show()


In [ ]:
esg_scores